In [1]:
import boto3
import configparser
import json
import pandas

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY=config.get('AWS','KEY')
SECRET=config.get('AWS','SECRET')

DWH_CLUSTER_TYPE=config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES=config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE=config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER=config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB=config.get("DWH","DWH_DB")
DWH_DB_USER=config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD=config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT=config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME=config.get("DWH", "DWH_IAM_ROLE_NAME")


In [3]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                     )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [4]:
# Create IAM Role
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)

An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhIamRole already exists.


In [5]:
# Attach IAM Policy
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

200

In [6]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

arn:aws:iam::384090142032:role/dwhIamRole


In [7]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [10]:

def prettyRedshiftProps(props):
    pandas.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pandas.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.chwhgyodub2m.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-5a97f522
7,NumberOfNodes,4


In [11]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT: " + DWH_ENDPOINT + "\n" + "DWH_ROLE_ARN: " + DWH_ROLE_ARN)

DWH_ENDPOINT: dwhcluster.chwhgyodub2m.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN: arn:aws:iam::384090142032:role/dwhIamRole


In [12]:
# Update clusters security group
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-fa928ea5')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists
